In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer
import requests, time, json
import time
import plotly.graph_objects as go
import pandas as pd

# BTCUSDT

In [ ]:
fig = go.FigureWidget()
fig.layout.title = 'BTCUSD'
fig.add_scatter()
fig

In [ ]:
kafka_topic_name = "ChemEng"
kafka_bootstrap_servers = 'localhost:9092'
kafka_bootstrap_servers = 'xxxxxxxxxxx.compute.amazonaws.com:xxxxxxxxx'

producer = KafkaProducer(bootstrap_servers=[kafka_bootstrap_servers],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

time_start = time.time()
time_count = 0
ind = 0
df = pd.DataFrame()
while time_count<30:
    time_current = time.time()
    prices = json.loads(requests.get("https://api.binance.com/api/v3/ticker/price").content)
    for i in prices:
        if i['symbol'] =='BTCUSDT':
            data = {'time':time.strftime('%d/%m/%y [%H:%M:%S]'), 'price':i['price']}
            data = dumps(data)
            data = str(ind)+':'+str(i['price'])
            # data = i['price']
#             print(time.strftime('%d/%m/%y [%H:%M:%S]'),i['price'])
            producer.send(kafka_topic_name, value=data)
            producer.flush()

            df = df.append(pd.DataFrame({'price': i['price'], 'time':time_count}, index=[0]))
            fig.data[0].x = df.time
            fig.data[0].y = df.price

    ind+=1     
    time_count = time_current-time_start
print("producer send done!",ind)

# ASSUME base on SHIB-USD

In [ ]:
dp = pd.read_csv("SHIB-USD.csv")
r1 = dp['Close'].to_list()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.lineplot(list(range(1,len(r1)+1)), r1)
plt.show()

In [ ]:
fig = go.FigureWidget()
fig.layout.title = 'Assume_price'
fig.add_scatter()
fig

In [ ]:
kafka_topic_name = "ChemEng"
kafka_bootstrap_servers = 'localhost:9092'
kafka_bootstrap_servers = 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'

producer = KafkaProducer(bootstrap_servers=[kafka_bootstrap_servers],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

# r1 = [i for i in range(100,200,10)]
# r2 = [i for i in range(1000,1200,10)]
# r1.extend(r2)
df = pd.DataFrame()
ind = 0
for i in r1:
    data = str(ind)+':'+str(i)
    producer.send(kafka_topic_name, value=data)
    producer.flush()
#     print(ind,i)
    df = df.append(pd.DataFrame({'price': i, 'time':ind}, index=[0]))
    fig.data[0].x = df.time
    fig.data[0].y = df.price
    ind+=1
    time.sleep(0.1)
print("producer send done!",ind)